# 位置が近すぎるラーメン屋を集約して点数の高い方を残す

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('./store.csv')
df.head()

,name,score,station,tel,address,latitude,longitude,open_time,closed_days,sheets,url,icon_url
0,銀座 八五（はちごう）,3.9,東銀座駅,050-5596-6139,東京都中央区銀座3-14-2 第一はなぶさビル 1F,35.670537,139.770138,11：00~15：0017：00~21：00※スープがなくなり次第、営業終了とさせていただい...,［毎週水曜日］［第2第4木曜日］,6席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13228602/,https://tblg.k-img.com/resize/640x640c/restaur...
1,麺屋ひょっとこ 交通会館店,3.78,有楽町駅,03-3211-6002,東京都千代田区有楽町2-10-1 東京交通会館 B1F,35.675160,139.764343,[平日]11:00～20:00[土]11:00～19:00※売り切れ次第終了,日・祝,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130102/13046637/,https://tblg.k-img.com/resize/640x640c/restaur...
2,むぎとオリーブ 銀座店,3.78,東銀座駅,03-3571-2123,東京都中央区銀座6-12-12 銀座ステラビル 1F,35.668956,139.764284,【平日】11:30～22:30 (LO22:00)【土・日・祝】11:00～22:30 (L...,NaN,15席 （カウンター15席）,https://tabelog.com/tokyo/A1301/A130101/13164932/,https://tblg.k-img.com/resize/640x640c/restaur...
3,銀座 朧月（おぼろづき）,3.78,銀座駅,03-6280-6166,東京都中央区銀座6-3-5 第一高橋ビル 1F,35.671581,139.761402,【月〜日】昼の部：11:30～15:30夜の部：17:30～22:00※スープがなくなり次第終了,不定休,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13118141/,https://tblg.k-img.com/resize/640x640c/restaur...
4,新橋 纏（まとい）,3.77,新橋駅,03-3436-6003,東京都港区新橋3-13-2 IKENO-5 1F,35.665866,139.756298,【平日】11:00～15:0017:30～22:00【土】11:00～15:0017:30～...,無休,9席 （カウンター席のみ）,https://tabelog.com/tokyo/A1301/A130103/13156937/,https://tblg.k-img.com/resize/640x640c/restaur...


In [11]:
df['x'] = 1
df2 = pd.merge(df, df, on='x', how='inner', suffixes=['', '_'])[
    ['url', 'score', 'latitude', 'longitude', 
     'url_', 'score_', 'latitude_', 'longitude_']]
df2['distance'] = (df2['latitude'] - df2['latitude_']) ** 2 + (df2['longitude'] - df2['longitude_'])  ** 2
del df['x']
df2.head()

,url,score,latitude,longitude,url_,score_,latitude_,longitude_,distance
0,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670537,139.770138,0.000000
1,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130102/13046637/,3.78,35.675160,139.764343,0.000055
2,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130101/13164932/,3.78,35.668956,139.764284,0.000037
3,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130101/13118141/,3.78,35.671581,139.761402,0.000077
4,https://tabelog.com/tokyo/A1301/A130101/13228602/,3.9,35.670537,139.770138,https://tabelog.com/tokyo/A1301/A130103/13156937/,3.77,35.665866,139.756298,0.000213


In [12]:
# 距離が近い店舗の中で最大のスコアを持つ店舗を抽出
for i, threshold in enumerate([0.00001, 0.0001]):
    tmp = df2[df2['distance'] < threshold].groupby(['url', 'score'], as_index=False)[['score_']].max()
    tmp = tmp[tmp['score'] == tmp['score_']]
    df[f'summarize_{i}'] = (df['url'].isin(set(tmp['url']))).astype(int)
df.head()

,name,score,station,tel,address,latitude,longitude,open_time,closed_days,sheets,url,icon_url,summarize_0,summarize_1
0,銀座 八五（はちごう）,3.9,東銀座駅,050-5596-6139,東京都中央区銀座3-14-2 第一はなぶさビル 1F,35.670537,139.770138,11：00~15：0017：00~21：00※スープがなくなり次第、営業終了とさせていただい...,［毎週水曜日］［第2第4木曜日］,6席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13228602/,https://tblg.k-img.com/resize/640x640c/restaur...,1,1
1,麺屋ひょっとこ 交通会館店,3.78,有楽町駅,03-3211-6002,東京都千代田区有楽町2-10-1 東京交通会館 B1F,35.675160,139.764343,[平日]11:00～20:00[土]11:00～19:00※売り切れ次第終了,日・祝,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130102/13046637/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
2,むぎとオリーブ 銀座店,3.78,東銀座駅,03-3571-2123,東京都中央区銀座6-12-12 銀座ステラビル 1F,35.668956,139.764284,【平日】11:30～22:30 (LO22:00)【土・日・祝】11:00～22:30 (L...,NaN,15席 （カウンター15席）,https://tabelog.com/tokyo/A1301/A130101/13164932/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
3,銀座 朧月（おぼろづき）,3.78,銀座駅,03-6280-6166,東京都中央区銀座6-3-5 第一高橋ビル 1F,35.671581,139.761402,【月〜日】昼の部：11:30～15:30夜の部：17:30～22:00※スープがなくなり次第終了,不定休,7席 （カウンターのみ）,https://tabelog.com/tokyo/A1301/A130101/13118141/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0
4,新橋 纏（まとい）,3.77,新橋駅,03-3436-6003,東京都港区新橋3-13-2 IKENO-5 1F,35.665866,139.756298,【平日】11:00～15:0017:30～22:00【土】11:00～15:0017:30～...,無休,9席 （カウンター席のみ）,https://tabelog.com/tokyo/A1301/A130103/13156937/,https://tblg.k-img.com/resize/640x640c/restaur...,1,0


In [13]:
df.to_csv('./store_summarize.csv', index=False)